In [1]:
import pandas as pd

In [2]:
df_games = pd.read_csv('../data/games.csv')
df_users = pd.read_csv('../data/user_ratings.csv')

# 1. Filtrar juegos con MaxPlayers y MinPlayers menor a dos

In [3]:
ans = df_games.shape[0]
df_games = df_games[df_games.MaxPlayers >= 2]
print(f'Diff {df_games.shape[0] - ans}')

Diff -538


In [4]:
ans = df_games.shape[0]
df_games = df_games[df_games.MinPlayers >= 2]
print(f'Diff {df_games.shape[0] - ans}')

Diff -3334


# 2. Filtrar 100 juegos por ranking

In [5]:
ans = df_games.shape[0]
df_games['mask'] = (df_games.loc[:,
                    ['Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts', 'Rank:familygames', 'Rank:thematic',
                     'Rank:cgs',
                     'Rank:wargames', 'Rank:partygames', 'Rank:childrensgames']] < 100).sum(axis=1) != 0
df_games = df_games[df_games['mask']].copy()

print(f'Diff {df_games.shape[0] - ans}')

Diff -17475


In [6]:
df_games.shape

(578, 49)

In [7]:
df_users.shape

(18942215, 3)

In [8]:
df_users = df_users[df_users.BGGId.isin(df_games.BGGId)].copy()

In [9]:
df_users.shape

(5286623, 3)

In [10]:
df_users.Username.value_counts()

Username
leffe dubbel      504
oldgoat3769967    426
TomVasel          413
Tolkana           404
Doel              402
                 ... 
Roald               1
randycread          1
cynycgirl           1
ScottandKellie      1
jobirnbaum          1
Name: count, Length: 309968, dtype: int64

# 3. Filtrar usuarios con menos de 20 reseñas

In [11]:
ans = df_users.shape[0]
usert_count = df_users.Username.value_counts()
usert_count = usert_count[usert_count > 20]
df_users = df_users[df_users.Username.isin(usert_count.index)].copy()
print(f'Diff {df_users.shape[0] - ans}, {df_users.shape[0]}, {ans}')

Diff -1453027, 3833596, 5286623


In [12]:
df_users.Username.value_counts().min()

21

In [14]:
unique_games = df_users.BGGId.unique()
ans = df_games.shape[0]
df_games = df_games[df_games.BGGId.isin(unique_games)]
print(f'Diff {df_games.shape[0] - ans}, {df_games.shape[0]}, {ans}')


Diff 0, 578, 578


# 4. Separar entrenamiento y test

In [15]:
df_users.reset_index(inplace=True, drop=True)

df_users['isTest'] = False
idxs = df_users.groupby('Username').sample(n=10).index
df_users.loc[idxs, 'isTest'] = True

print(df_users.isTest.sum())

778670


# 5. Save

In [16]:
df_games.to_csv('../data/df_games_train.csv', index=False)
df_users.to_csv('../data/df_users_train.csv', index=False)